## 제2유형_연습하기_iris 종 분류

## 데이터 분석 순서
- 1. 라이브러리 및 데이터 확인
  2. 데이터 탐색(EDA)
  3. 데이터 전처리 및 분리
  4. 모델링 및 성능평가
  5. 예측값 제출

## 1. 라이브러리 및 데이터 확인

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()
x = pd.DataFrame(iris.data, columns=['sepal_length','sepal_width','petal_length','petal_width'])
y = iris.target # 'setosa'=0, 'versicolor'=1, 'virginica'=2
y = np.where(y>0, 1, 0) # setosa 종은 0, 나머지 종은 1로 변경

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=2023)
x_test = pd.DataFrame(x_test)
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
y_train.columns = ['species']

# 결측치 삽입
x_test.loc[0, 'sepal_length'] = None
x_train.loc[0, 'sepal_length'] = None
# 이상치 삽입
x_train.loc[0, 'sepal_width'] = 150

## 붓꽃의 종(Species)을 분류해보자
- 데이터의 결측치, 이상치에 대해 처리하고
- 분류모델을 사용하여 정확도, F1 score, AUC 값을 산출하시오.
- 제출은 result 변수에 담아 양식에 맞게 제출하시오.

In [11]:
# 데이터 설명
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

## 2. 데이터 탐색(EDA)

In [2]:
# 데이터의 행/열 확인
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(120, 4)
(31, 4)
(120, 1)


In [14]:
# 초기 데이터 확인
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

    sepal_length  sepal_width  petal_length  petal_width
2            4.7          3.2           1.3          0.2
49           5.0          3.3           1.4          0.2
66           5.6          3.0           4.5          1.5
     sepal_length  sepal_width  petal_length  petal_width
93            5.0          2.3           3.3          1.0
69            5.6          2.5           3.9          1.1
137           6.4          3.1           5.5          1.8
   species
0        0
1        0
2        1


In [15]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 2 to 44
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  119 non-null    float64
 1   sepal_width   120 non-null    float64
 2   petal_length  120 non-null    float64
 3   petal_width   120 non-null    float64
dtypes: float64(4)
memory usage: 8.8 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 93 to 0
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  30 non-null     float64
 1   sepal_width   30 non-null     float64
 2   petal_length  30 non-null     float64
 3   petal_width   30 non-null     float64
dtypes: float64(4)
memory usage: 2.3 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   species  120 non-n

In [16]:
# x_train과 x_test 데이터의 기초통계량을 잘 비교해보세요
print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe().T)

              count      mean        std  min    25%   50%    75%    max
sepal_length  119.0  5.916807   0.845759  4.3  5.150  6.00  6.500    7.9
sepal_width   120.0  4.292500  13.419277  2.2  2.800  3.00  3.325  150.0
petal_length  120.0  3.816667   1.798848  1.1  1.575  4.40  5.225    6.9
petal_width   120.0  1.226667   0.780512  0.1  0.300  1.35  1.800    2.5
              count      mean       std  min    25%   50%    75%  max
sepal_length   30.0  5.576667  0.705488  4.6  5.000  5.50  5.900  7.6
sepal_width    30.0  3.000000  0.522593  2.0  2.625  3.00  3.300  4.2
petal_length   30.0  3.523333  1.631518  1.0  1.600  4.05  4.925  6.6
petal_width    30.0  1.090000  0.685490  0.1  0.350  1.15  1.575  2.3
         count      mean       std  min  25%  50%  75%  max
species  120.0  0.666667  0.473381  0.0  0.0  1.0  1.0  1.0


In [17]:
# y데이터도 구체적으로 살펴보세요
print(y_train.head(3))

   species
0        0
1        0
2        1


In [18]:
# y데이터도 구체적으로 살펴보세요
print(y_train.value_counts())

species
1          80
0          40
Name: count, dtype: int64


## 3. 데이터 전처리 및 분리

### 1)결측치, 2)이상치, 3)변수 처리하기

In [19]:
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
sepal_length    1
sepal_width     1
petal_length    1
petal_width     1
dtype: int64
species    0
dtype: int64


In [20]:
# 결측치 제거
#df = df.dropna()
#df.dropna().shape

In [21]:
# 결측치 대체(평균값, 중앙값, 최빈값)
# 연속형 변수 : 중앙값, 평균값
#df['변수명'].median()
#df['변수명'].mean()

# 범주형 변수 : 최빈값
#df['변수명'].mode()

#df['변수명'] = df['변수명'].fillna(대체할 값)

In [22]:
# 결측치 대체(중앙값)
## ** 주의사항 : train 데이터의 중앙값으로 test 데이터도 변경해줘야 함 **
median = x_train['sepal_length'].median()
x_train['sepal_length'] = x_train['sepal_length'].fillna(median)
x_test['sepal_length'] = x_test['sepal_length'].fillna(median)

In [23]:
print(x_test.isnull().sum())

sepal_length    0
sepal_width     1
petal_length    1
petal_width     1
dtype: int64


In [25]:
median_sepal_width = x_train['sepal_width'].median()
x_test['sepal_width'] = x_test['sepal_width'].fillna(median_sepal_width)
median_petal_length = x_train['petal_length'].median()
x_test['petal_length'] = x_test['petal_length'].fillna(median_petal_length)
median_petal_width = x_train['petal_width'].median()
x_test['petal_width'] = x_test['petal_width'].fillna(median_petal_width)

In [26]:
print(x_test.isnull().sum())

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64


In [28]:
# 이상치 확인
cond1 = (x_train['sepal_width']>10)
print(len(x_train[cond1]))

1


In [33]:
# 이상치 대체
# 예를 들어 'sepal_width' 값이 10이 넘으면 이상치라고 가정해본다면
# 이상치를 제외한 Max 값을 구해서 대체해보자
cond1 = (x_train['sepal_width']<=10)
Max = x_train['sepal_width'][cond1].max()
print(Max)
x_train['sepal_width'] = np.where(x_train['sepal_width']>=10, Max, x_train['sepal_width'])
print(x_train.describe())

4.4
       sepal_length  sepal_width  petal_length  petal_width
count    120.000000   120.000000    120.000000   120.000000
mean       5.917500     3.079167      3.816667     1.226667
std        0.842232     0.428383      1.798848     0.780512
min        4.300000     2.200000      1.100000     0.100000
25%        5.175000     2.800000      1.575000     0.300000
50%        6.000000     3.000000      4.400000     1.350000
75%        6.500000     3.325000      5.225000     1.800000
max        7.900000     4.400000      6.900000     2.500000


In [34]:
# 변수처리
# 불필요한 변수 제거
#df = df.drop(columns=['변수1','변수2'])
#df = df.drop(['변수1','변수2'], axis=1)

# 필요시 변수 추가(파생변수 생성)
#df['파생변수명'] = df['A']*df['B']

# 원핫인코딩(가변수 처리)
#x_train = pd.get_dummies(x_train)
#x_test = pd.get_dummies(x_test)

### 데이터 분리

In [35]:
# 데이터를 훈련 세트와 검증용 세트로 분할 (80% 훈련, 20% 검증용)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train['species'],
                                                  test_size=0.2,
                                                  stratify=y_train['species'],
                                                  random_state=2023)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(96, 4)
(24, 4)
(96,)
(24,)


## 4. 모델링 및 성능평가

In [36]:
# 랜덤포레스트 모델 사용 (참고 : 회귀모델은 RandomForestRegressor)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [37]:
# 모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [38]:
# 모델 성능 평가 (accuracy, f1 score, AUC 등)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
acc = accuracy_score(y_val, y_pred) # (실제값, 예측값)
f1 = f1_score(y_val, y_pred) # (실제값, 예측값)
# 다중분류일 경우 f1 = f1_score(y_val, y_pred, average='macro')
auc = roc_auc_score(y_val, y_pred)

In [39]:
# 정확도 (Accuracy)
print(acc)

1.0


In [40]:
# F1 score
print(f1)

1.0


In [41]:
# AUC
print(auc)

1.0


In [42]:
# 참고사항
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred) # (실제값, 예측값)
print(cm)

[[ 8  0]
 [ 0 16]]


In [43]:
#       예측
#       0  1
#실제 0 TN FP
#실제 1 FN TP

## 5. 예측값 제출

### (주의) x_test를 모델에 넣어 나온 예측값을 제출해야 함

In [44]:
# 모델을 사용하여 테스트 데이터 예측
# 1. 특정 클래스로 분류할 경우 (predict)
y_result = model.predict(x_test)
print(y_result[:5])
# 2. 특정 클래스로 분류될 확률을 구할 경우 (predict_proba)
y_result_prob = model.predict_proba(x_test)
print(y_result_prob[:5])

# 이해해보기
result_prob = pd.DataFrame({
    'result':y_result,
    'prob_0':y_result_prob[:,0]
})
# setosa 일 확률 : y_result_prob[:,0]
# 그 외 종일 확률 : y_result_prob[:,1]
print(result_prob[:5])

[1 1 1 0 1]
[[0.01 0.99]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.01 0.99]]
   result  prob_0
0       1    0.01
1       1    0.00
2       1    0.00
3       0    1.00
4       1    0.01


In [45]:
# 데이터를 저장한 다음 불러와서 제대로 제출했는지 확인
pd.DataFrame({'result':y_result}).to_csv('0011.csv', index=False)

In [46]:
df2 = pd.read_csv('0011.csv')
print(df2.head())

   result
0       1
1       1
2       1
3       0
4       1
